In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as :

    SETTING = yaml.load()

PATH = make_path_dict(SETTING)

In [ ]:
gene_x_sample = pd.read_csv(PATH["gene_x_sample.processed.tsv"], sep="\t", index_col=0)

target_x_sample = pd.read_csv(PATH["target_x_sample.tsv"], sep="\t", index_col=0)

gene_sets = kraft.read_gmts(SETTING["gene_set_file_paths"])

gene_set_x_information = pd.read_csv(
    PATH["gene_set_x_information.tsv"], sep="\t", index_col=0
)

In [ ]:
for target_name in target_x_sample.index:

    output_directory_path = os.path.join(PATH["gsea/"], target_name)

    kraft.establish_path(output_directory_path, "directory")

    score_moe_p_value_fdr = pd.read_csv(
        os.path.join(
            PATH["find_differentially_expressed_gene/"],
            target_name,
            "all.{}.tsv".format(SETTING["gsea_match_function_name"]),
        ),
        sep="\t",
        index_col=0,
    )

    gene_score = score_moe_p_value_fdr["Score"]

    gene_score.name = "Gene Score"

    gene_set_score = (
        kraft.single_sample_gseas(
            gene_score.to_frame(),
            gene_sets.loc[
                gene_set_x_information.index[
                    gene_set_x_information["Good"].astype(bool)
                ]
            ],
            statistic=SETTING["gsea_statistic"],
            n_job=SETTING["n_job"],
        )
        .squeeze()
        .sort_values()
    )

    gene_set_score.name = "GSEA Score ({})".format(SETTING["gsea_statistic"])

    gene_set_score.to_csv(
        os.path.join(
            output_directory_path, "{}.tsv".format(SETTING["gsea_match_function_name"])
        ),
        sep="\t",
    )

    print(gene_set_x_information.loc[gene_set_score.index])

    kraft.plot_point_and_annotate(
        None,
        gene_set_score,
        "y",
        annotation=(("Peek", SETTING["gene_sets_to_peek"], 8, "#20d9ba"),),
        title={"text": target_name},
        html_file_path=os.path.join(
            output_directory_path,
            "{}.plot_point_and_annotate.html".format(
                SETTING["gsea_match_function_name"]
            ),
        ),
    )

    output_directory_path = os.path.join(
        output_directory_path, "{}.mountain".format(SETTING["gsea_match_function_name"])
    )

    kraft.establish_path(output_directory_path, "directory")

    for gene_set_name in SETTING["gene_sets_to_peek"]:

        kraft.single_sample_gsea(
            gene_score,
            gene_sets.loc[gene_set_name],
            statistic=SETTING["gsea_statistic"],
            title="{}<br>{}".format(target_name, gene_set_name),
            html_file_path=os.path.join(
                output_directory_path, "{}.html".format(gene_set_name)
            ),
        )